# Combine Orbit and Omni Data for Future Investigations

In [1]:
import pandas as pd
import sys
import os
import glob

In [2]:
#add read_io module to current path ()
file_path = 'D:\\GitHub\\DataIO\\'
sys.path.append(os.path.dirname(file_path))

import data_io

In [3]:
orb, _ = data_io.image_orb(sdate='2001',local_dir='E:/Data/IMAGE')

In [4]:
omni, _ = data_io.load_omni(sdate=orb['DateTime'].min(), edate=orb['DateTime'].max(), res='5m')

In [8]:
pro_f = glob.glob('E:/Data/IMAGE/processed/*/*/*')
im_t = pd.to_datetime([r[-15:-4] for r in pro_f], format='%Y%j%H%M')
im_t = im_t.set_names('DateTime').to_frame()
im_t['filename'] = pro_f

In [16]:
type(pro_f)

list

In [10]:
tol = pd.Timedelta('2.5m')

In [12]:
database = pd.merge_asof(left=im_t,right=omni,
                         left_index=True,right_on='DateTime',
                         direction='nearest',tolerance=tol, 
                         suffixes=['_dat','_omni'])

database = pd.merge_asof(left=database, right=orb,
                         left_index=True,right_on='DateTime',
                         direction='nearest',tolerance=tol, 
                         suffixes=['_dat','_orb'])

In [13]:
database.to_hdf(r'E:/Data/IMAGE/image_orb_omni.hdf',key='database',complevel=3)